Создание директории для данных с опечатками

In [ ]:
import re 
import os

def AddDir(rootDir):
        for lists in os.listdir(rootDir):
                path = os.path.join(rootDir, lists)
                new_dir = re.sub("corrupted_date","annotated-corpus-corrupted",path)
                os.mkdir(new_dir)
                print(new_dir)
AddDir('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\corrupted_date\\')

Токенизация данных с опечатками

In [30]:
from tokenize import Token
from nltk.stem import PorterStemmer
import nltk
from nltk.stem import WordNetLemmatizer
import regex as re
import os
import glob

email = '[a-zA-Z0-9]+@\w+[\.\w+]+'
money='\$[0-9]+[\.[0-9]+]?[k]?'
date='[0-9]{1,2}\s[a-zA-Z]+\s[0-9]{1,4}|[0-9]{1,2}\s[0-9]{1,2}\s[0-9]{1,4}|[0-9]{1,2}\S[0-9]{1,2}\S[0-9]{1,4}|[a-zA-Z]+\s[0-9]{1,2}\S?\s[0-9]{1,4}'
phone='\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}'
words='[a-zA-z]+[\'[a-z]+]?|w+|\S'
prop_names=''
full_exp=email+'|'+money+'|'+date+'|'+phone+'|'+words

path_change='D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus-corrupted\\'

ps =PorterStemmer()
lemmatizer = WordNetLemmatizer()
def DataProcess(rootDir):
    for lists in os.listdir(rootDir):
                    path = os.path.join(rootDir, lists)
                    for filename in glob.glob(path+'\\*'):
                        with open(os.path.join(os.getcwd(), filename), 'r') as file:
                            
                            txt = file.read().rstrip()

                            dtxt=re.sub(r"\.\s|\.\n",".天官赐福,百无禁忌%@!^&ЁЪЫї",txt)
                            etxt=re.sub(r"\!\s|\!\n","!天官赐福,百无禁忌%@!^&ЁЪЫї",dtxt)
                            qtxt=re.sub(r"\?\s|\?\n","?天官赐福,百无禁忌%@!^&ЁЪЫї",etxt)
                            sentences= re.split(r'\天\官\赐\福\,\百\无\禁\忌\%\@\!\^\&\Ё\Ъ\Ы\ї', qtxt)

                            tokenize_regex = re.compile(full_exp, re.I)
                            new_path = re.sub("corrupted_date","annotated-corpus-corrupted",file.name)

                            with open(new_path+".tsv", "w") as f:
                                for s in sentences:
                                    tokens =tokenize_regex.findall(s)
                                    for t in tokens:
                                        f.write(t)
                                        f.write('\t')
                                        f.write(ps.stem(t))
                                        f.write('\t')
                                        f.write(lemmatizer.lemmatize(t))
                                        f.write('\n')
                                    f.write("\n")

In [31]:
DataProcess('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\corrupted_date\\')

Создание словаря

In [2]:
import os

def get_file_tokens(path):
    tokens = []
    with open(path, "r") as f:
        line = f.readline()
        while line:
            if line:
                token = line.split("\t")[0]
                if token != '\n':
                    tokens.append(token)
            line = f.readline()
    f.close()
    return tokens

In [33]:
import os
import glob

def GetAllTokens(rootDir):
    tokens=[]
    for lists in os.listdir(rootDir):
                    path = os.path.join(rootDir, lists)
                    for filename in glob.glob(path+'\\*'):
                       tokens = tokens + get_file_tokens(filename)
    return tokens


In [34]:
tokens = GetAllTokens('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus\\test\\')

In [3]:
def write_dict(path,t):
    with open(path, "w") as f:
        for i in t:
            f.write(i)
            f.write('\n')
    f.close()

In [38]:
from collections import OrderedDict

dict = list(OrderedDict.fromkeys(tokens))

In [39]:
write_dict("dictionary.csv", dict)

In [4]:
def read_dict(path):
    dictionary = []
    with open(path, "r") as f:
        while True:
            line=f.readline()
            if not line:
                break
            dictionary.append(line.replace('\n',''))
    return dictionary

In [5]:
dictionary=read_dict("dictionary.csv")

Поиск опечаток

In [6]:
import numpy as np

keyboard_array_mapping = [
    ['`~', '1!', '2@', '3#', '4$', '5%', '6^', '7&', '8*', '9(', '0)', '-_', '=+'],
    ['qQ', 'wW', 'eE', 'rR', 'tT', 'yY', 'uU', 'iI', 'oO', 'pP', '[{', ']}', '\\|'],
    ['aA', 'sS', 'dD', 'fF', 'gG', 'hH', 'jJ', 'kK', 'lL', ';:', '\'"'],
    ['zZ', 'xX', 'cC', 'vV', 'bB', 'nN', 'mM', ',<', '.>', '/?'],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
]


def is_symbol_in_keyboard_array(symbol, array):
    return True in [symbol in row for row in array]


def get_keyboard_symbol_location(symbol, array):
    for array_row in array:
        if symbol in array_row:
            row = array.index(array_row)
            column = array_row.index(symbol)
            return row, column
    raise ValueError("{} not found".format(symbol))


def get_keyboard_distance(symbol1, symbol2):
    coord1 = get_keyboard_symbol_location(symbol1, keyboard_array_mapping)
    coord2 = get_keyboard_symbol_location(symbol2, keyboard_array_mapping)
    return np.sqrt(np.square(coord1[0] - coord2[0]) + np.square(coord1[1] - coord2[1]))


def get_max_keyboard_distance():
    get_keyboard_distance(keyboard_array_mapping[0][0], keyboard_array_mapping[-1][-1])

In [7]:
removal_cost = 1
insertion_cost = 1
substitution_cost = 2
no_op_cost = 0


def get_weighted_cost(replace_cost, symbol1, symbol2):
    try:
        return replace_cost + (get_keyboard_distance(symbol1, symbol2) / get_max_keyboard_distance())
    except ValueError:
        return substitution_cost


def count_edit_distance(word1, word2):
    n = len(word1) + 1
    m = len(word2) + 1
    matrix = np.zeros((n, m))
    for i in range(1, n):
        matrix[i][0] = i * removal_cost
    for j in range(1, m):
        matrix[0][j] = j * insertion_cost
    for i in range(1, n):
        for j in range(1, m):
            matrix[i, j] = min(
                matrix[i - 1][j] + removal_cost,
                matrix[i][j - 1] + insertion_cost,
                matrix[i - 1][j - 1] + (get_weighted_cost(
                    substitution_cost - 1,
                    word1[i - 1],
                    word2[j - 1])
                )
                if word1[i - 1] != word2[j - 1] else no_op_cost)
    return matrix[n - 1, m - 1]


def fix_token_typo(token, tokens_dictionary_list):
    if token in tokens_dictionary_list:
        return token
    min_distance = 10000
    fixed_token = ""
    for correct_token in tokens_dictionary_list:
        distance = count_edit_distance(token, correct_token)
        if distance < min_distance:
            min_distance = distance
            fixed_token = correct_token
    return fixed_token

In [8]:
import os
import re 

def print_score(valid_tokens_in_corrupted_file_cnt,
                correct_tokens_in_corrupted_file_before_cnt,
                correct_tokens_in_corrupted_file_after_cnt):
    print("Общее количество токенов: {}".format(valid_tokens_in_corrupted_file_cnt))
    print("Количество не содержащее опечатки до обработки: {}".format(correct_tokens_in_corrupted_file_before_cnt))
    print("Количество не содержащее опечатки после обработки: {}".format(correct_tokens_in_corrupted_file_after_cnt))
    print("До обработки: {}"
          .format(correct_tokens_in_corrupted_file_before_cnt / valid_tokens_in_corrupted_file_cnt))
    print("После: {}"
          .format(correct_tokens_in_corrupted_file_after_cnt / valid_tokens_in_corrupted_file_cnt))


def handle_corrupted_file(path_to_corrupted_tokens_dataset, path_to_correct_tokens_dataset):
    valid_tokens_in_corrupted_file_cnt = 0
    correct_tokens_in_corrupted_file_before_cnt = 0
    correct_tokens_in_corrupted_file_after_cnt = 0

    token_dict = dictionary
    fixed_token_cnt = 1
    token_counter_limit = 200
    for root, dirs, files in os.walk(path_to_corrupted_tokens_dataset):
        if fixed_token_cnt == token_counter_limit:
            break
        for file in files:
            if fixed_token_cnt == token_counter_limit:
                break
            corrupted_token_list = get_file_tokens(root + '\\' + file)
            corrupted_token_list_cnt = len(corrupted_token_list)
            original_token_list = get_file_tokens(
                root.replace(path_to_corrupted_tokens_dataset, path_to_correct_tokens_dataset+'\\') + '\\' + file)
            original_token_list_cnt = len(original_token_list)
            if corrupted_token_list_cnt != original_token_list_cnt:
                print("Skip: {}".format(root + '/' + file))
            else:
                valid_tokens_in_corrupted_file_cnt += corrupted_token_list_cnt
                for index, token in enumerate(corrupted_token_list):
                    if fixed_token_cnt == token_counter_limit:
                        break
                    token = token.split()[0]
                    fixed_token = fix_token_typo(token, token_dict)
                    if fixed_token == token:
                        correct_tokens_in_corrupted_file_before_cnt += 1
                    else:
                        corrupted_token_list[index] = fixed_token
                    fixed_token_cnt += 1
                for index, token in enumerate(corrupted_token_list):
                    if corrupted_token_list[index] == original_token_list[index]:
                        correct_tokens_in_corrupted_file_after_cnt += 1
    print_score(
        valid_tokens_in_corrupted_file_cnt,
        correct_tokens_in_corrupted_file_before_cnt,
        correct_tokens_in_corrupted_file_after_cnt)

In [9]:
handle_corrupted_file('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus-corrupted\\','D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus\\test')

Skip: D:\S\NLP\nlp-22-autumn\projects\shiyan_v3\assets\annotated-corpus-corrupted\alt.atheism/53068.tsv
Skip: D:\S\NLP\nlp-22-autumn\projects\shiyan_v3\assets\annotated-corpus-corrupted\alt.atheism/53257.tsv
Skip: D:\S\NLP\nlp-22-autumn\projects\shiyan_v3\assets\annotated-corpus-corrupted\alt.atheism/53260.tsv
Общее количество токенов: 280
Количество не содержащее опечатки до обработки: 176
Количество не содержащее опечатки после обработки: 223
До обработки: 0.6285714285714286
После: 0.7964285714285714
